In [2]:
from typing import Tuple
import re
import json
from collections import defaultdict, OrderedDict

In [3]:
import numpy as np
import pandas as pd

## Read namuwiki dump

In [4]:
with open('../data/namuwiki.json') as f:
    context_ = json.load(f)

In [5]:
context = defaultdict(dict)
for doc in context_:
    context[doc['title']][doc['namespace']] = doc

## Parser

In [28]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('\|\|(.+?)\|\|(.+?)\|\|')
regex_bracket = re.compile('\((.+?)\)')

regex_tags = OrderedDict({
    'regex_del': (r'\1', re.compile('~~(.+?)~~')),
    'regex_dash': (r'\1', re.compile('--(.+?)--')),
    'regex_color': (r'\2', re.compile('\{\{\{\#(.+?) \'\'\'(.+?)\'\'\' \}\}\}')),
    'regex_link': (r'\1', re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')),
    'regex_attach': ('attach', re.compile('\[\[파일:(.+?)\]\]')),
    'regex_document': (r'\1', re.compile('\[\[(.[^:]+?)\]\]')),
    
    'regex_tag': ('', re.compile('\<(.+?)\>')),
    'regex_comment': ('', re.compile('\[\*(.+?)\]')),
})

In [7]:
def parse(text: str) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) -> Tuple[str, int]:
        return tag.subn(target, text)
    
    count = 0
    
    while True:
        for target_tag in regex_tags.values():
            text, count = _parse(text, *target_tag)
            if count:
                break
        if not count:
            break
            
    return text.strip()

## Get drama titles from channles

In [8]:
interested = ["JTBC 금토 드라마(2014~2017)", "JTBC 금토 드라마(2017~2020)", "JTBC 드라마", "JTBC 수목 드라마", "JTBC 월화 드라마(2011~2014)", "JTBC 월화 드라마(2017~2020)", "JTBC 주말 드라마", "KBS 수목 드라마(2001~2005)", "KBS 수목 드라마(2006~2010)", "KBS 수목 드라마(2011~2015)", "KBS 수목 드라마(2016~2020)", "KBS 월화 드라마(2001~2005)", "KBS 월화 드라마(2006~2010)", "KBS 월화 드라마(2011~2015)", "KBS 월화 드라마(2016~2020)", "KBS 학교 시리즈", "MBC 수목 미니시리즈(2006~2010)", "MBC 수목 미니시리즈(2011~2015)", "MBC 수목 미니시리즈(2016~2020)", "MBC 아침 드라마(2011~2015)", "MBC 아침 드라마(2016~2020)", "MBC 예능 드라마", "MBC 월화 미니시리즈(2006~2010)", "MBC 월화 미니시리즈(2016~2020)", "MBC 월화특별기획(2011~2015)", "MBC 일일 드라마(2016~2020)", "MBC 일일 연속극(2011~2015)", "MBC 주말 드라마(2011~2015)", "MBC 주말 드라마(2016~2020)", "MBC 주말 특별기획(2011~2015)", "MBC 주말 특별기획(2016~2020)", "MBC 하이킥 시리즈", "MBN 수목 드라마", "OCN 로맨스 드라마", "OCN 수목 오리지널", "OCN 오리지널 드라마(2010~2016)", "OCN 월화 오리지널", "OCN 토일 오리지널(2017~2020)", "SBS 금토 드라마(2019~현재)", "SBS 드라마 스페셜(1992~1995)", "SBS 드라마 스페셜(1996~2000)", "SBS 드라마 스페셜(2001~2005)", "SBS 드라마 스페셜(2006~2010)", "SBS 드라마 스페셜(2011~2015)", "SBS 드라마 스페셜(2016~2020)", "SBS 아침 연속극(2016~2020)", "SBS 월화 드라마(1991~1995)", "SBS 월화 드라마(1996~2000)", "SBS 월화 드라마(2001~2005)", "SBS 월화 드라마(2006~2010)", "SBS 월화 드라마(2011~2015)", "SBS 월화 드라마(2016~2020)", "TV CHOSUN 토일드라마", "tvN 금요 드라마(2007~2015)", "tvN 금토 드라마", "tvN 로맨스가 필요해 시리즈", "tvN 불금 시리즈(2017~)", "tvN 월화 드라마(2011~2015)", "tvN 월화 드라마(2016~2020)", "tvN 토일 드라마(2017~2020)"]

In [9]:
titles = set()

In [10]:
for inter in interested:
    document = context[inter]
    matches = regex_document.findall(document['1']['text'])
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        titles.add(name)

## Get metadata from document

In [11]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [12]:
data = defaultdict(dict)
notexists = []
notparser = []

In [16]:
for title in titles:
    try:
        if '(드라마)' not in title and f'{title}(드라마)' in context:
            document = context[title]['0']['text']
        else:
            document = context[title]['0']['text']
    except KeyError:
        notexists.append(title)
        
    matches = regex_table.findall(document)
    
    d = defaultdict(str)
    
    for key, value in matches:
        key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
        if key:
            key = key.strip()
            d[key] = f'{d[key]} {value}'
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [17]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

125 pages are not parsable
2 pages are not exists


## Create table from data

In [75]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        regex_bracket.sub('', title).replace(' ', '').strip(), *tuple(map(parse, map(lambda c: values[c], table_columns)))
    ])))

In [76]:
df = pd.DataFrame(table)
df.columns = ['제목', *table_columns]

### Parse datetime

In [77]:
regex_date = re.compile('(.+?)년(.*)월(.*)일')

In [78]:
date_start = []
for index, date in enumerate(df['방송 기간']):
    ds, *de = map(regex_date.findall, map(str.strip, date.split('~' if '~' in date else '-')))
    ds, *_ = ds or ['unknown']
    try:
        date_start.append(pd.datetime(*tuple(map(int, ds))))
    except (ValueError, TypeError):
        date_start.append('unknown')
    
assert len(date_start) == np.size(df, 0)

In [79]:
df['방송 시작'] = pd.Series(date_start)

## Show Dataframe

In [80]:
print(df.shape)
df.head()

(736, 9)


,제목,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자,방송 시작
0,비밀의숲,2017년 6월 10일 ~ 2017년 7월 30일,16부작 + 스페셜 1부작,"드라마, 스릴러, 범죄",tvN,"씨그널 엔터테인먼트 그룹, IOK 미디어",이수연(작가) 이수연(작가),"조승우, 배두나, 이준혁, 유재명, 신혜선 外",2017-06-10 00:00:00
1,붉은달푸른해,2018년 11월 21일 ~ 2019년 1월 16일,32부작 + 스페셜 1부작,"스릴러, 수사물",문화방송,메가몬스터]라는 CJ E&M 계열회사였으나 2017년 카카오에 매각되었다. ] ...,도현정,"김선아, 이이경, 남규리, 엔(빅스) 外",2018-11-21 00:00:00
2,시를잊은그대에게,2018년 3월 26일 ~ 2018년 5월 15일,16부작,,tvN,tvN,"명수현, 백선우, 최보림 등","이유비, 이준혁, 장동윤 外",2018-03-26 00:00:00
3,메리대구공방전,2007년 5월 16일 ~ 2007년 7월 5일,16부작,"드라마, 로맨틱 코미디",문화방송,,김인영(각본가),"지현우, 이하나, 왕빛나, 이민우(배우) 外",2007-05-16 00:00:00
4,안녕하세요하느님,2006년 1월 9일 ~ 2006년 2월 28일,16부작,"멜로, 드라마",KBS 2TV,"김종학프로덕션, 포도나무프로덕션",강은경,"유건, 김옥빈, 이종혁(배우) 外",2006-01-09 00:00:00


In [81]:
df.to_csv('../results/namuwiki.csv', index=None)